In [31]:
import pandas as pd
import numpy as np

In [2]:
dat = pd.read_csv('data/project_db_500.csv',low_memory=False)
dat.columns

Index(['script_ID', 'project_ID', 'sprite-type', 'sprite-name', 'script-rank',
       'coordinates', 'total-blocks', 'p_ID', 'project-name', 'username',
       'total-views', 'total-remixes', 'total-favorites', 'total-loves',
       'is-remix', 'Script_ID', 'block-rank', 'block-type', 'param1', 'param2',
       'param3', 'param4', 'param5', 'param6', 'param7', 'param8', 'param9',
       'param10', 'param11', 'param12', 'param13', 'param14', 'param15',
       'param16', 'param17', 'param18', 'param19', 'param20', 'param21',
       'param22', 'param23', 'param24', 'project_ID.1', 'Abstraction',
       'Parallelism', 'Logic', 'Synchronization', 'FlowControl',
       'UserInteractivity', 'DataRepresentation', 'Mastery', 'Clones',
       'CustomBlocks', 'InstancesSprites'],
      dtype='object')

In [42]:
#replacing dashes with underscores
def clean(dat):
    dat.columns = [i.replace('-', '_') for i in dat.columns]
    #dat = dat.drop(columns=['Script_ID','p_ID'])
    dat = dat[dat.columns[~dat.columns.str.contains('param')]]
    dat = dat.drop(columns=['sprite_type','sprite_name','script_rank','coordinates','username','project_ID.1'])
    return dat

can certainly drop some of the columns, especially the redundant ones (and probably some others as well)
redundant: 
    script_ID and Script_ID
    project_ID and p_ID
will consider removing teh "param*" columns, since we're not really concerned with the pars put into the blocks

In [63]:
dat_train = pd.read_csv('data/project_db_1000.csv',low_memory=False)
dat_train = clean(dat)
dat_predict = pd.read_csv('data/project_db_500.csv',low_memory=False)
dat_predict = clean(dat)

dat_train = pd.get_dummies(dat_train)


In [64]:
label = np.array(dat_train['total_favorites'])
features = dat_train.drop('total_favorites',axis=1)
feature_list = list(features.columns)
features = np.array(features)

In [65]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [71]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)
test_labels.mean()

Training Features Shape: (50001, 580)
Training Labels Shape: (50001,)
Testing Features Shape: (16668, 580)
Testing Labels Shape: (16668,)


12.409587233021359

In [67]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 10 decision trees
rf = RandomForestRegressor(n_estimators = 10, random_state = 42)# Train the model on training data
rf.fit(train_features, train_labels);

In [68]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)# Calculate the absolute errors
errors = abs(predictions - test_labels)# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 5))

Mean Absolute Error: 3e-05


In [69]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 100.0 %.


In [62]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot# Pull out one tree from the forest
tree = rf.estimators_[5]# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot# Pull out one tree from the forest
tree = rf.estimators_[5]# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')# Write graph to a png file
graph.write_png('tree.png')

ModuleNotFoundError: No module named 'pydot'